In [1]:
import pandas as pd
import os
from pathlib import Path
import gc
import multiprocessing
from joblib import Parallel, delayed
from tqdm import tqdm

import cudf
import numba
from numba import cuda
import numpy as np
import numerapi

# parallelize the process on all columns using joblib
import multiprocessing
from joblib import Parallel, delayed
from tqdm import tqdm

from src.data_utils import (
    load_recent_data_from_file,
    save_daily_data,
    save_in_folders,
    get_latest_date,
    read_available_dates
)
from src.config import (
    DAILY_DATA_DIR,
    DATA_DIR,
    DAILY_PRIMARY_FEATURES_DIR,
    DAILY_SECONDARY_FEATURES_DIR,
    DAILY_SCALED_FEATURES_DIR,
)


In [2]:
MERGED_HISTORICAL_DATA_LOCATION = os.path.join(DATA_DIR, "merged_data_historical.parquet")
LIVE_DATA_LOCATION = os.path.join(DATA_DIR, "merged_data_live.parquet")

In [3]:
ml_data = pd.read_parquet(MERGED_HISTORICAL_DATA_LOCATION)
live_data = pd.read_parquet(LIVE_DATA_LOCATION)

#ml_data = ml_data.groupby("date").filter(lambda x: len(x) > 500)

In [4]:
ml_data

,bloomberg_ticker,friday_date,data_type,target_4d,target_20d,target_20d_raw_return,target_20d_factor_neutral,target_20d_factor_feat_neutral,date,feature_1_sma_5,...,feature_2_ratio_rsi_50_100,feature_2_ratio_rsi_50_200,feature_2_ratio_rsi_100_200,feature_2_ratio_rsi_5_close,feature_2_ratio_rsi_10_close,feature_2_ratio_rsi_20_close,feature_2_ratio_rsi_50_close,feature_2_ratio_rsi_100_close,feature_2_ratio_rsi_200_close,date_str
0,1332 JP,20030131.0,train,0.75,0.50,0.75,0.50,0.50,2003-01-31,4,...,3,3,2,0,0,0,0,0,0,2003-01-31
1,1801 JP,20030131.0,train,0.50,0.75,0.75,0.50,0.50,2003-01-31,4,...,4,4,1,0,0,0,0,0,0,2003-01-31
2,1802 JP,20030131.0,train,0.25,0.50,0.75,0.75,0.50,2003-01-31,4,...,4,4,2,0,0,0,0,0,0,2003-01-31
3,1803 JP,20030131.0,train,0.50,0.50,0.50,0.50,0.50,2003-01-31,4,...,1,1,2,0,0,0,0,0,0,2003-01-31
4,1812 JP,20030131.0,train,0.25,0.50,0.50,0.50,0.50,2003-01-31,4,...,0,0,0,0,0,0,0,0,0,2003-01-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3954907,ZUMZ US,20230602.0,validation,0.50,1.00,1.00,1.00,1.00,2023-06-02,1,...,2,2,1,2,2,3,3,3,3,2023-06-02
3954908,ZUO US,20230602.0,validation,0.50,0.25,0.25,0.25,0.50,2023-06-02,1,...,1,3,4,4,4,4,3,3,3,2023-06-02
3954909,ZURN SW,20230602.0,validation,0.50,0.50,0.50,0.50,0.25,2023-06-02,4,...,4,4,1,1,1,0,0,0,0,2023-06-02
3954910,ZWS US,20230602.0,validation,0.50,0.75,0.75,0.75,0.75,2023-06-02,1,...,4,4,4,3,3,3,3,3,2,2023-06-02


In [7]:
from sklearn.base import BaseEstimator, RegressorMixin, ClassifierMixin

In [9]:
class BaselineModel(RegressorMixin):
    def __init__(self, model):
        self.model = model

    def fit(self, X, y):
        self.model.fit(X, y)
        return self

    def predict(self, X):
        return self.model.predict(X)

# 1. Provided Train-validation splits

In [10]:
train_set = ml_data[ml_data["data_type"] == "train"]
val_set = ml_data[ml_data["data_type"] == "validation"]

In [ ]:
bas